In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


In [180]:
# Specify the URL of the Gap Bestsellers website
url1 = "https://www.gap.com/browse/category.do?cid=3024420&nav=meganav%3AMen%3AJust%20Arrived%3ABestsellers"
url = "https://www.gap.com/browse/category.do?cid=13658#pageId=0&mlink=5643,20012060,DP_1_W_Dresses"

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-card")))

# Scroll down to load all products (you may need to adjust the number of scrolls)
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.CLASS_NAME, "loading-spinner")))

# Get the page source after scrolling
page_source = driver.page_source

# Create BeautifulSoup object
soup = BeautifulSoup(page_source, 'html.parser')

# Find all divs with the specified class
product_divs = soup.find_all('div', class_='cat_product-image category-page-mvez9e')

# Initialize lists to store extracted data
product_links = []
product_names = []

# Iterate through product divs
for product_div in product_divs:
    # Find the anchor tag inside the div
    anchor_tag = product_div.find('a')
    
    # Extract link
    product_link = anchor_tag['href']
    product_links.append(product_link)
    
    # Extract product name
    product_name = anchor_tag.img['alt']
    product_names.append(product_name)

# Create a dictionary with the data
data = {
    'Product Name': product_names,
    'Product Link': product_links
}

# Create a DataFrame from the dictionary
product_catalogue = pd.DataFrame(data)

# Initialize an empty DataFrame to store the review data
df = pd.DataFrame(columns=["product_url", "product_name", "review_heading", "review_description", "rating"])

# Loop through each product URL
for product in product_links:
    driver.get(product)
    index = product_links.index(product)

    # Scroll down to load reviews (you may need to adjust the scroll and sleep times)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(15)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # Get the page source after scrolling
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Find review headings, descriptions, and ratings
    review_headings = []
    review_headers = soup.find_all('h2', attrs={'class': 'pr-rd-review-headline'})
    for header in review_headers:
        text = header.get_text()
        review_headings.append(text)

    review_descriptions = []
    review_desc = soup.find_all('p', attrs={'class': "pr-rd-description-text", 'lang' : "en"})
    for desc in review_desc:
        text = desc.get_text()
        review_descriptions.append(text)

    ratings = []
    stars = soup.find_all('div', attrs={'class': 'pr-rd-star-rating'})
    for star in stars:
        text = star.get_text()
        ratings.append(text)

    # Create a DataFrame for the current product's reviews
    product_df = pd.DataFrame({
        "product_name": [product_names[index]] * len(review_headings),
        "review_heading": review_headings,
        "review_description": review_descriptions,
        "rating": ratings,
        "product_url": [product] * len(review_headings)
    })

    # Append the product's reviews to the main DataFrame
    df = pd.concat([df, product_df], ignore_index=True)

# Close the web driver
driver.close()
driver.quit()

In [171]:
df.to_csv('GAP_Bestsellers_Women_Reviews.csv', index=False)